In [14]:
import pandas as pd
import numpy as np

In [15]:
companies_selected = pd.read_excel("Companies.xlsx")

In [16]:
# since TSR is in % we will grade wight in % as well
grade_weight = {"A": 90, "B": 80, "C": 70, "D": 60, "E": 50, "F": 40}
companies_selected["GradeWeight"] = companies_selected["Grade"].map(grade_weight)

In [17]:
def calculate_grade_score(data, x):
    return (1 - x) * data["TSR"] + x * data["GradeWeight"]

In [ ]:
# default values set as: 16% for industry allocation
# beta value for potfolio should be below 1
def monte_carlo_simulation(
    data,
    max_industry_allocation=0.16,
    target_beta=1.0,
    iterations=100,
):

    best_portfolio = None
    best_portfolio_beta = float("inf")

    for _ in range(iterations):
        # we first want to normalise the TSR:
        data["Industry TSR total"] = data.groupby("Industry")["TSR"].transform("sum")
        data["weighted TSR"] = data["TSR"] / data["Industry TSR total"]
        # This will give us cases where our grade is not the main factor and TSR is which provides us with different scores on multiple iterations
        x = np.random.uniform(0.6, 0.85)

        # execute the function mention above
        data["Grade Score"] = data.apply(calculate_grade_score, axis=1, args=(x,))

        # obtain a weight scross all the investments. Basically getting a ratio so values for the weights will be between 0 and 1
        # basic normalisation
        data["Weight"] = data["Grade Score"] / data["Grade Score"].sum()

        # Lets focus on each industry
        # what is the current weight allocated to each industry
        data["Industry Allocation"] = data.groupby("Industry")["Weight"].transform(
            "sum"
        )
        data["Allocation_by_restriction"] = np.where(
            data["Industry Allocation"] > max_industry_allocation,
            max_industry_allocation,
            data["Industry Allocation"],
        )

        data["Normalised Weight"] = np.where(
            data["Allocation_by_restriction"] == 0.16,
            (data["Weight"] / data["Industry Allocation"]) * 0.16,
            data["Weight"],
        )

        # NEED TO ADD WEIGHT FOR THE MAX ALLOCATION

        # Calculate Portfolio Beta
        portfolio_beta = (data["Beta Value"] * data["Normalised Weight"]).sum()

        if portfolio_beta < target_beta:
            best_portfolio_beta = portfolio_beta
            break

    return best_portfolio_beta, data

In [19]:
final_beta, ideal_portfolio = monte_carlo_simulation(companies_selected)

0.8408685313614467


In [20]:
final_beta

0.939976061158996

In [21]:
ideal_portfolio["Normalised Weight"].sum()

0.9575959423195852

In [22]:
ideal_portfolio.groupby("Industry")["Normalised Weight"].sum()

Industry
Consumer Staples    0.103264
Energy              0.120818
Finance             0.132375
Healthcare          0.132739
Industruals         0.093360
Real Estate         0.106296
Technology          0.160000
Utilities           0.108744
Name: Normalised Weight, dtype: float64

In [23]:
ideal_portfolio["final_weight"] = (
    ideal_portfolio["Normalised Weight"] / ideal_portfolio["Normalised Weight"].sum()
) * 100

In [24]:
round(ideal_portfolio["final_weight"].sum(), 2) == 100.00

True

In [25]:
max(ideal_portfolio["final_weight"])

5.930841638377342

In [26]:
ideal_portfolio.sort_values("final_weight", ascending=False).head(15)

,Company Name,Industry,Beta Value,Grade,TSR,GradeWeight,Industry TSR total,weighted TSR,Grade Score,Weight,Industry Allocation,Allocation_by_restriction,Normalised Weight,final_weight
44,NVDA,Technology,1.76,C,2694.86,70,4829.52,0.557997,487.697827,0.071845,0.202404,0.160000,0.056793,5.930842
40,TSLA,Technology,2.12,A,1236.11,90,4829.52,0.255949,272.382168,0.040126,0.202404,0.160000,0.031720,3.312411
25,Eli Lilly and Company (LLY),Healthcare,0.42,C,675.16,70,1435.04,0.470482,166.300000,0.024498,0.132739,0.132739,0.024498,2.558332
13,Raymond James Financial (RJF),Finance,1.00,A,196.37,90,1525.19,0.128751,106.926814,0.015752,0.132375,0.132375,0.015752,1.644945
11,LPL Financial (LPLA),Finance,0.85,C,276.95,70,1525.19,0.181584,102.932257,0.015163,0.132375,0.132375,0.015163,1.583493
17,Morgan Stanley (MS),Finance,1.34,B,219.69,80,1525.19,0.144041,102.229075,0.015060,0.132375,0.132375,0.015060,1.572676
20,UnitedHealth Group Incorporated (UNH),Healthcare,0.59,A,159.18,90,1435.04,0.110924,101.008715,0.014880,0.132739,0.132739,0.014880,1.553902
15,Goldman Sachs (GS),Finance,1.35,B,211.02,80,1525.19,0.138357,100.849405,0.014857,0.132375,0.132375,0.014857,1.551451
10,Interactive Brokers (IBKR),Finance,0.79,D,303.51,60,1525.19,0.198998,98.750104,0.014547,0.132375,0.132375,0.014547,1.519156
30,The Williams Companies,Energy,1.06,C,248.45,70,1085.06,0.228974,98.397011,0.014495,0.120818,0.120818,0.014495,1.513724
